In [1]:
import pickle

In [2]:
# add path
import sys

sys.path.append('..')

In [3]:
from reasoners.visualization import visualize

/home/shibo/anaconda3/envs/rap/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
with open("/home/shibo/LLM-search/logs/bw_MCTS/08272023-151944/algo_output/1.pkl", 'rb') as f:
    mcts_result = pickle.load(f)

In [7]:
from reasoners.visualization.tree_snapshot import NodeData, EdgeData
from reasoners.algorithm.mcts import MCTSNode

def blocksworld_node_data_factory(n: MCTSNode) -> NodeData:
    return NodeData({"block state": n.state.blocks_state if n.state else "Not expanded",
                     "# goals satisfied": int(n.reward_details["goal_reached"][1]) if hasattr(n, "reward_details") else "N/A"})
def blocksworld_edge_data_factory(n: MCTSNode) -> EdgeData:
    return EdgeData({"Q": n.Q, "intuition": n.fast_reward_details["intuition"], "self_eval": n.fast_reward_details["self_eval"], "action": n.action})

In [8]:
visualize(mcts_result, node_data_factory=blocksworld_node_data_factory, edge_data_factory=blocksworld_edge_data_factory)

Visualizer URL: https://www.llm-reasoners.net/visualizer/d7fc478d-12f8-4b64-b366-5816c072207d?accessKey=2bc2ac61


In [35]:
from reasoners.visualization import TreeLog

In [41]:
import os
os.mkdir("/home/shibo/LLM-search/logs/bw_MCTS/07282023-161026/tree_logs")
for i in range(1, 21):
    with open(f"/home/shibo/LLM-search/logs/bw_MCTS/07282023-161026/algo_output/{i}.pkl", 'rb') as f:
        mcts_result = pickle.load(f)
    tree_log = TreeLog.from_mcts_results(mcts_result, node_data_factory=blocksworld_node_data_factory, edge_data_factory=blocksworld_edge_data_factory)
    with open(f"/home/shibo/LLM-search/logs/bw_MCTS/07282023-161026/tree_logs/tree_log_{i}.json", 'w') as f:
        f.write(str(tree_log))

In [5]:
import json
questions = []
with open("/home/shibo/LLM-search/logs/bw_MCTS/07282023-161026/questions.jsonl", "r") as f:
    data = f.readlines()
    for i in range(20):
        d = json.loads(data[i])
        question = "My goal is to have " + d["goal"] + " The initial state is that, " + d["init"] + " What's the action plan to achieve my goal?"
        questions.append(question)

json.dump(questions, open("bw_questions.json", "w"))